<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Creating Azure SQL Servers with Python

## Azure Actions Covered

* Creating and listing Azure SQL Servers
* Creating and listing databases on a SQL Server
* Creating and listing firewall rules for your SQL Server
* Connecting to an Azure SQL Server with `pyodbc`

In this lecture, we're going to learn how to set up Azure SQL Servers with Python.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. AZURE_SUBSCRIPTION_ID). We'll add some new imports as well.

In [1]:
from azure.identity import AzureCliCredential
# New imports
from azure.mgmt.sql import SqlManagementClient
from azure.mgmt.sql.models import FirewallRule

# Module for connecting to SQL Server
import pyodbc

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

We'll instantiate our credential as well as our `SqlManagementClient`.

In [2]:
credential = AzureCliCredential()
sql_client = SqlManagementClient(credential, AZURE_SUBSCRIPTION_ID)

## Azure SQL Server

Using our SQL Management Client, we can create a new SQL Server. We need to pass the resource group and set a server name as well as certain server parameters, such as:
* `location` - Azure location for the SQL Server
* `version` - Version of SQL Server to install
* `administrator_login` - User name of admin user
* `adminstrator_login_password` - Password for the admin user

For a full list of parameters, see [Server class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-sql/azure.mgmt.sql.models.server?view=azure-python).

In [3]:
sql_server = sql_client.servers.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server',
    parameters={
        'location': DEFAULT_LOCATION,
        'version': '12.0',
        'administrator_login': 'benadmin',
        'administrator_login_password': 'testpassword123!'
    }
)
sql_server.wait()

Let's list our SQL Servers to make sure it was created.

In [4]:
for server in sql_client.servers.list():
    print(server.name)

bens-server


## Databases

Now that we've created the SQL Server, we can create a new database on the server using the SQL client. You'll need to pass parameters such as:
* `location` - Azure location.
* `sample_name` - Name of sample schema to use to populate database.
* `edition` - Edition component of the SKU, such as `Basic`, `Standard`, and `Premium`.
* `compute_model` - Compute model of the database.
* `family` - Generation component of the SKU, such as `Gen4` or `Gen5`.
* `capacity` - The capacity component of the SKU.

For a full list of parameters, see the [Database class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-sql/azure.mgmt.sql.models.database?view=azure-python).

In [5]:
database = sql_client.databases.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server',
    database_name='bens-db',
    parameters={
        'location': DEFAULT_LOCATION,
        'sample_name': 'AdventureWorksLT',
        'edition': 'GeneralPurpose',
        'compute_model': 'Serverless',
        'family': 'Gen5',
        'capacity': 2
    }
)
database.wait()

Now let's list all the databases in our SQL Server.

In [6]:
for db in sql_client.databases.list_by_server(DEFAULT_RESOURCE_GROUP, 'bens-server'):
    print(db.name)

master
bens-db


## Connect to SQL Server

To connect to the SQL Server, we'll need to create a connection string to pass to `pyodbc`. You'll also need to install the ODBC Driver for SQL Server for your operating system:

* [Windows](https://learn.microsoft.com/en-us/sql/connect/odbc/windows/microsoft-odbc-driver-for-sql-server-on-windows?view=sql-server-ver16)
* [macOS](https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/install-microsoft-odbc-driver-sql-server-macos?view=sql-server-ver16)
* [Linux](https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=alpine18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline)

In [24]:
SERVER = 'bens-server.database.windows.net'
DATABASE = 'bens-db'
USERNAME = 'benadmin'
PASSWORD = 'testpassword123!'
# Has to be installed
DRIVER = '{ODBC Driver 18 for SQL Server}'

connection_string = f'DRIVER={DRIVER};PORT=1433;SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

We can pass the connection string to try to connect to the SQL Server and see what happens.

In [ ]:
# connection = pyodbc.connect(connection_string)
# cursor = connection.cursor()

Our connection will be rejected because we haven't created a firewall rule to allow traffic from our computer/Jupyter Notebook.

## Firewall Rules

Let's use the SQL client to create a firewall rule. We can pull the relevant IP address from our error message. For a full list of paramters that can be passed, see the [FirewallRule class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-sql/azure.mgmt.sql.models.firewallrule?view=azure-python).

In [26]:
firewall = sql_client.firewall_rules.create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server',
    firewall_rule_name='JupyterNotebook',
    parameters={
        'start_ip_address': '72.175.52.98',
        'end_ip_address': '72.175.52.98'
    }
)


Let's list all the firewall rules on our SQL Server as well.

In [14]:
for fr in sql_client.firewall_rules.list_by_server(DEFAULT_RESOURCE_GROUP, 'bens-server'):
    print(fr.name)

AllowMyIp


Now let's try to connect to the server again, and everything should work.

In [27]:
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()